# 장애인콜택시 대기시간 예측
## 단계1.데이터 전처리

## 0.미션

* 1.데이터를 탐색하며 정보 획득
    * 데이터는 2015-01-01 ~ 2022-12-31 까지의 서울 장애인 콜택시 운행 정보입니다. 우리는 평균대기시간을 예측하고자 합니다. 
    * 요일, 월, 계절, 연도별 운행 정보에 어떤 주기와 변화가 있는지 탐색해 봅시다.
* 2.분석을 위한 데이터 구조 만들기
    * 문제정의 : 
        * 전 날 콜택시 운행이 종료되었을 때, 다음 날 대기시간을 예측하고자 합니다.

        * 만약 다음 날 대기시간을 예측할 수 있다면, 일정 범위 내에서 배차를 조절할 수 있을 뿐만 아니라, 향후 교통약자의 이동 편의 증진을 위한 정책 수립 및 개선에 기여할 수 있습니다. 
    * 이를 위한 데이터 구조를 만들어 봅시다.
        * 분석 단위는 일별 데이터 입니다.
        * 주어진 데이터 : 장애인 콜택시 운행 정보, 서울시 날씨
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전 날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
        * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙여서 만듭시다.
        * 휴무일 데이터는 패키지를 통해서 다운받아 사용합니다.
    * Feature Engineering
        * 대기시간에 영향을 주는 요인을 도출하고(가설수립) 이를 feature로 생성합시다.
        * 주어진 그대로의 데이터가 아닌 새로운 feature를 생성해 봅시다.
            * 날짜와 관련된 feature : 요일, 월, 계절 ... 
            * 시계열 특성이 반영된 feature : 최근 7일간의 평균 대기시간 ...




## 1.환경설정

* 세부 요구사항
    - 경로 설정 : 다음의 두가지 방법 중 하나를 선택하여 폴더를 준비하고 데이터를 로딩하시오.
        * 1) 로컬 수행(Ananconda)
            * 제공된 압축파일을 다운받아 압축을 풀고
            * anaconda의 root directory(보통 C:/Users/< ID > 에 project 폴더를 만들고, 복사해 넣습니다.
        * 2) 구글콜랩
            * 구글 드라이브 바로 밑에 project 폴더를 만들고, 
            * 데이터 파일을 복사해 넣습니다.
    - 라이브러리 설치 및 로딩
        * requirements.txt 파일로 부터 라이브러리 설치
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다. 
        * 필요하다고 판단되는 라이브러리를 추가하세요.

### (1) 경로 설정

#### 1) 로컬 수행(Anaconda)
* project 폴더에 필요한 파일들을 넣고, 본 파일을 열었다면, 별도 경로 지정이 필요하지 않습니다.

In [4]:
# path = 'C:/Users/User/project/'

#### 2) 구글 콜랩 수행

* 구글 드라이브 연결

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# path = '/content/drive/MyDrive/project/'

### (2) 라이브러리 설치 및 불러오기

#### 1) 설치

* requirements.txt 파일을 아래 위치에 두고 다음 코드를 실행하시오.
    * 로컬 : 다음 코드셀 실행
    * 구글콜랩 : requirements.txt 파일을 왼쪽 [파일]탭에 복사해 넣고 다음 코드셀 실행

In [7]:
# !pip install -r requirements.txt

#### 2) 라이브러리 로딩

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from datetime import datetime
# 더 필요한 라이브러리를 아래에 추가합니다.



### (3) 데이터 불러오기
* 주어진 데이터셋
    * 장애인 콜택시 운행 정보 : open_data.csv
    * 날씨 데이터 : weather.csv

#### 1) 데이터로딩

In [9]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
weather = pd.read_csv('./weather.csv')

In [37]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
taxi = pd.read_csv('./open_data.csv')

In [39]:
max(taxi['평균대기시간'])

96.1

In [40]:
max(taxi['평균승차거리'])

14136

In [41]:
max(taxi['평균요금'])

2733

In [38]:
# 아래에 실습코드를 작성하고 결과를 확인합
taxi.head()

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


In [14]:
taxi.head()

,기준일,차량운행,접수건,탑승건,평균대기시간,평균요금,평균승차거리
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


#### 2) 기본 정보 조회

In [15]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 전체 데이터의 행,열 개수 확인

weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4018 entries, 0 to 4017
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             4018 non-null   object 
 1   temp_max         4018 non-null   float64
 2   temp_min         4018 non-null   float64
 3   rain(mm)         4018 non-null   float64
 4   humidity_max(%)  4018 non-null   float64
 5   humidity_min(%)  4018 non-null   float64
 6   sunshine(MJ/m2)  4018 non-null   float64
dtypes: float64(6), object(1)
memory usage: 219.9+ KB


In [16]:
taxi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2922 entries, 0 to 2921
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   기준일     2922 non-null   object 
 1   차량운행    2922 non-null   int64  
 2   접수건     2922 non-null   int64  
 3   탑승건     2922 non-null   int64  
 4   평균대기시간  2922 non-null   float64
 5   평균요금    2922 non-null   int64  
 6   평균승차거리  2922 non-null   int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 159.9+ KB


In [17]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 전체 데이터의 모든 변수 확인

weather.columns

Index(['Date', 'temp_max', 'temp_min', 'rain(mm)', 'humidity_max(%)',
       'humidity_min(%)', 'sunshine(MJ/m2)'],
      dtype='object')

In [18]:
taxi.columns

Index(['기준일', '차량운행', '접수건', '탑승건', '평균대기시간', '평균요금', '평균승차거리'], dtype='object')

#### 3) 칼럼이름을 영어로 변경
* 꼭 필요한 작업은 아니지만, 데이터를 편리하게 다루고 차트에서 불필요한 경고메시지를 띄우지 않게 하기 위해 영문으로 변경하기를 권장합니다.


In [19]:
# 아래에 실습코드를 작성하고 결과를 확인합니다
taxi.rename(columns={'기준일': 'Date', 
                     '차량운행': 'TaxiCount', 
                     '접수건': 'Reservation', 
                     '탑승건': 'RidingCount', 
                     '평균대기시간': 'WaitingTime', 
                     '평균요금':'FareAvg', 
                     '평균승차거리': 'DistanceAvg'}, inplace=True)

In [20]:
taxi.head()

,Date,TaxiCount,Reservation,RidingCount,WaitingTime,FareAvg,DistanceAvg
0,2015-01-01,213,1023,924,23.2,2427,10764
1,2015-01-02,420,3158,2839,17.2,2216,8611
2,2015-01-03,209,1648,1514,26.2,2377,10198
3,2015-01-04,196,1646,1526,24.5,2431,10955
4,2015-01-05,421,4250,3730,26.2,2214,8663


## 2.데이터 기본 탐색

* **세부요구사항**
    * 날짜 요소에 따라 각 정보의 패턴을 조회 합니다.
        * 일별, 요일별, 주차별, 월별, 연도별
        * 접수건, 탑승건, 거리, 요금, 대기시간 등
    * 제시된 범위 외에 가능하다면 추가 탐색을 시도합니다.

### (1) 주기별 분석을 위해서 날짜 변수 추가하기
* data를 복사합니다.
* 복사한 df에 요일, 주차, 월, 연도 등을 추가합니다.

In [21]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df = taxi.copy()

In [22]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df['Date'] = pd.to_datetime(df['Date'])

In [23]:
weather['Date'] = pd.to_datetime(weather['Date'])

In [24]:
df = pd.merge(weather, df)
df.head()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),TaxiCount,Reservation,RidingCount,WaitingTime,FareAvg,DistanceAvg
0,2015-01-01,-4.3,-9.8,0.0,52.0,33.0,9.79,213,1023,924,23.2,2427,10764
1,2015-01-02,-2.0,-8.9,0.0,63.0,28.0,9.07,420,3158,2839,17.2,2216,8611
2,2015-01-03,2.4,-9.2,0.0,73.0,37.0,8.66,209,1648,1514,26.2,2377,10198
3,2015-01-04,8.2,0.2,0.0,89.0,58.0,5.32,196,1646,1526,24.5,2431,10955
4,2015-01-05,7.9,-0.9,0.0,95.0,52.0,6.48,421,4250,3730,26.2,2214,8663


In [25]:
df['WaitingTime'] = df['WaitingTime'].shift(-1)

In [26]:
df.head()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),TaxiCount,Reservation,RidingCount,WaitingTime,FareAvg,DistanceAvg
0,2015-01-01,-4.3,-9.8,0.0,52.0,33.0,9.79,213,1023,924,17.2,2427,10764
1,2015-01-02,-2.0,-8.9,0.0,63.0,28.0,9.07,420,3158,2839,26.2,2216,8611
2,2015-01-03,2.4,-9.2,0.0,73.0,37.0,8.66,209,1648,1514,24.5,2377,10198
3,2015-01-04,8.2,0.2,0.0,89.0,58.0,5.32,196,1646,1526,26.2,2431,10955
4,2015-01-05,7.9,-0.9,0.0,95.0,52.0,6.48,421,4250,3730,23.6,2214,8663


In [27]:
df['Date'] = df['Date'].shift(-1)

In [28]:
df = df[:-1]
df.tail()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),TaxiCount,Reservation,RidingCount,WaitingTime,FareAvg,DistanceAvg
2916,2022-12-27,1.5,-7.9,0.0,85.0,46.0,8.27,603,5555,4605,44.4,2163,7889
2917,2022-12-28,3.0,-7.3,0.0,86.0,51.0,10.25,669,5635,4654,44.8,2198,8178
2918,2022-12-29,-0.3,-5.4,0.1,92.0,40.0,10.86,607,5654,4648,52.5,2161,7882
2919,2022-12-30,1.7,-7.8,0.0,71.0,34.0,10.88,581,5250,4247,38.3,2229,8433
2920,2022-12-31,2.1,-4.0,0.0,87.0,38.0,10.84,600,5293,4200,33.7,2183,8155


In [29]:
df.fillna(method='ffill', inplace=True)

In [30]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Week'] = df['Date'].dt.isocalendar().week
df['WeekDay'] = df['Date'].dt.weekday

In [31]:
weekday_map = {0: 'MON', 1: 'TUE', 2: 'WED', 3: 'THU', 4: 'FRI', 5: 'SAT', 6: 'SUN'}
df['WeekDay'] = df['WeekDay'].map(weekday_map)

In [32]:
df.head()

,Date,temp_max,temp_min,rain(mm),humidity_max(%),humidity_min(%),sunshine(MJ/m2),TaxiCount,Reservation,RidingCount,WaitingTime,FareAvg,DistanceAvg,Year,Month,Day,Week,WeekDay
0,2015-01-02,-4.3,-9.8,0.0,52.0,33.0,9.79,213,1023,924,17.2,2427,10764,2015,1,2,1,FRI
1,2015-01-03,-2.0,-8.9,0.0,63.0,28.0,9.07,420,3158,2839,26.2,2216,8611,2015,1,3,1,SAT
2,2015-01-04,2.4,-9.2,0.0,73.0,37.0,8.66,209,1648,1514,24.5,2377,10198,2015,1,4,1,SUN
3,2015-01-05,8.2,0.2,0.0,89.0,58.0,5.32,196,1646,1526,26.2,2431,10955,2015,1,5,2,MON
4,2015-01-06,7.9,-0.9,0.0,95.0,52.0,6.48,421,4250,3730,23.6,2214,8663,2015,1,6,2,TUE


### (3) 요일별

* 차량 운행수

In [35]:
df_weekday = df.groupby('WeekDay', as_index=False)['TaxiCount'].mean()

In [36]:
df_weekday

,WeekDay,TaxiCount
0,FRI,494.078947
1,MON,224.767386
2,SAT,485.107656
3,SUN,262.175060
4,THU,484.726619
5,TUE,485.014388
6,WED,495.076739


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

sns.barplot(x='WeekDay', y='TaxiCount', data=df_weekday)
plt.show()

* 접수건, 탑승건

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_weekday_rsv = df.groupby('WeekDay', as_index=False)['Reservation'].sum()
sns.barplot(x='WeekDay', y='Reservation', data=df_weekday_rsv)
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_weekday_riding = df.groupby('WeekDay', as_index=False)['RidingCount'].sum()
sns.barplot(x='WeekDay', y='RidingCount', data=df_weekday_riding)
plt.show()

In [ ]:
plt.plot(df_weekday_riding['WeekDay'], df_weekday_riding['RidingCount'], label='RidingCount')
plt.plot(df_weekday_rsv['WeekDay'], df_weekday_rsv['Reservation'], label='Reservation')
plt.legend()
plt.show()

* 대기시간

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

plt.scatter(df['Date'], df['WaitingTime'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_weekday_wait = df.groupby('WeekDay', as_index=False)['WaitingTime'].sum()

In [ ]:
sns.barplot(x='WeekDay', y='WaitingTime', data=df_weekday_wait)
plt.show()

* 운임

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

plt.scatter(df['Date'], df['FareAvg'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 이동거리

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

plt.scatter(df['Date'], df['DistanceAvg'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



### (2) 일별

* 차량 운행수

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

plt.scatter(df['Date'], df['TaxiCount'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 접수건, 탑승건

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

plt.scatter(df['Date'], df['Reservation'])

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 대기시간

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
plt.subplot(2, 1, 1)
sns.kdeplot(x= 'WaitingTime', data = df, hue = 'Year')
plt.subplot(2, 1, 2)
sns.boxplot(x='WaitingTime', data=df)
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 운임

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

sns.histplot(df['FareAvg'])

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 이동거리

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

sns.histplot(df['DistanceAvg'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



### (4) 월별

* 차량 운행수

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_month_taxi = df.groupby('Month', as_index=False)['TaxiCount'].sum()
plt.plot(df_month_taxi['Month'], df_month_taxi['TaxiCount'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 접수건, 탑승건

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_month_rsv = df.groupby('Month', as_index=False)['Reservation'].sum()
plt.plot(df_month_rsv['Month'], df_month_rsv['Reservation'])
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_month_riding = df.groupby('Month', as_index=False)['RidingCount'].sum()
plt.plot(df_month_riding['Month'], df_month_riding['RidingCount'],label='RidingCount')
plt.plot(df_month_rsv['Month'], df_month_rsv['Reservation'], label='Reservation')
plt.legend()
plt.show()

* 대기시간

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_month_wait = df.groupby('Month', as_index=False)['WaitingTime'].sum()
plt.plot(df_month_wait['Month'], df_month_wait['WaitingTime'])
# plt.plot(df_month_rsv['Month'], df_month_rsv['Reservation'], label='Reservation')
# plt.legend()
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 운임

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_month_fare = df.groupby('Month', as_index=False)['FareAvg'].sum()
plt.plot(df_month_fare['Month'], df_month_fare['FareAvg'])
# plt.plot(df_month_rsv['Month'], df_month_rsv['Reservation'], label='Reservation')
# plt.legend()
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
for i in range(2015, 2023):
    df_year_month = df.loc[df['Year'] == i].groupby(['Season'], as_index=False)['RidingCount'].sum()
    # plt.subplot(2, 1, 1)
    plt.plot(df_year_month['Season'], df_year_month['RidingCount'])
    # sns.histplot(df_year_month['Season'], df_year_month['TaxiCount'])
    plt.plot()
    plt.title(i)

    plt.show()

In [ ]:
# for i in range(2015, 2023):
#     df_year_month = df.loc[df['Year'] == i].groupby(['Season'], as_index=False)['rain(mm)'].sum()
#     # plt.subplot(2, 1, 1)
#     plt.plot(df_year_month['Season'], df_year_month['rain(mm)'])
#     # sns.histplot(df_year_month['Season'], df_year_month['TaxiCount'])
#     plt.plot()
#     plt.title(i)

#     plt.show()
df_humadity = df.groupby('humidity_max(%)', as_index=False)['WaitingTime'].sum()
plt.plot(df_humadity['humidity_max(%)'], df_humadity['WaitingTime'])
plt.plot()
plt.show()

* 이동거리

In [ ]:
df_month_distance = df.groupby('Month', as_index=False)['DistanceAvg'].sum()
# plt.plot(df_month_fare['Month'], df_month_fare['FareAvg'], label='Fare')
plt.plot(df_month_distance['Month'], df_month_distance['DistanceAvg'], label='Distance')
# plt.plot(df_month_rsv['Month'], df_month_rsv['Reservation'], label='Reservation')
# plt.legend()
plt.show()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



### (5) 연도별

* 차량 운행수

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
df_year_taxi = df.groupby('Year', as_index=False)['TaxiCount'].sum()
plt.plot(df_year_taxi['Year'], df_year_taxi['TaxiCount'])
plt.show()


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



* 접수건, 탑승건

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_year_rsv = df.groupby('Year', as_index=False)['Reservation'].sum()
plt.plot(df_year_rsv['Year'], df_year_rsv['Reservation'])
plt.show()


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df_year_riding = df.groupby('Year', as_index=False)['RidingCount'].sum()
plt.plot(df_year_riding['Year'], df_year_riding['RidingCount'])
plt.show()

## 3.데이터 구조 만들기

* **세부요구사항**
    * 조건 : 
        * 목표 : 전날 저녁, 다음날 평균 대기시간을 예측하고자 합니다.
        * 날씨 데이터는 실제 측정값이지만, 다음 날에 대한 예보 데이터로 간주합니다. 
            * 예를 들어, 
                * 2020-12-23 의 날씨 데이터는 전날(12월22일) 날씨예보 데이터로 간주하여 분석을 수행합니다.
                * 2020-12-22일의 장애인 이동 데이터로 23일의 대기시간을 예측해야 하며, 이때 고려할 날씨데이터는 23일 데이터 입니다.
    * 장애인 이동 데이터를 기준으로 날씨 데이터를 붙입니다.

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



### (1) target 만들기
* 예측하는 날짜, 대기시간(target)으로 기준을 잡습니다.

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 대기시간(waiting time)을 오늘의 데이터를 활용하여 예측 해야하는 대상(target)으로 설정

target = 'WaitingTime'

### (2) 날씨 데이터 붙이기
* merge를 활용합니다. 기준은 운행정보 입니다.

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# 익일의 실제 날씨 데이터를 전일에 발표된 예보데이터로 판단


In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



### (3) 새로운 feature를 생성해 봅시다.
* 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 그외 새로운 feature 도출 : 최소 2개 이상
    * 예 : 공휴일, 최근 7주일간의 평균 대기시간, 탑승률 등

#### 1) 날짜와 관련된 변수 추가하기 : 요일, 월, 계절, 연도
* 요일 이름, 계절이름, 월 이름으로 만드는 경우에는, 변수를 pd.Categorical로 범주형을 만들면서 순서를 지정하는 것이 이후 그래프를 그릴 때 순서대로 표현할 수 있습니다.


In [ ]:
df['WeekDay'] = df['Date'].dt.day_name()
df['WeekDay'] = pd.Categorical(df['WeekDay'], 
                                  categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'])
df['Month'] = df['Date'].dt.month

df['Season'] = np.where(df['Month'].isin([3,4,5]), 'Spring',
                           np.where(df['Month'].isin([6,7,8]), 'Summer',
                                    np.where(df['Month'].isin([9,10,11]), 'Fall', 'Winter')))
df['Season'] = pd.Categorical(df['Season'], categories=['Spring','Summer','Fall','Winter'])
                                     
df['Year'] = df['Date'].dt.year
# data0

In [ ]:
# df.drop('weekDay', inplace=True)

#### 2) 공휴일 정보
* workalendar 패키지를 설치하고, 대한민국 공휴일 정보를 끌어와 봅시다.

* 휴무일 데이터 패키지 설치

In [ ]:
# !pip install workalendar

* 간단 사용법

In [ ]:
from workalendar.asia import SouthKorea
cal = SouthKorea()
pd.DataFrame(cal.holidays(2023))

* 휴무일 데이터셋 만들기 2015 ~ 2022
* 실제로 휴무일에 해당하지만 workalendar 라이브러리에 없는 날짜는 직접 추가해봅시다.
    * 휴무일 장애인 콜택시의 접수건 변화에 대한 특징을 찾아 이를 바탕으로 데이터를 조회하여 찾아볼 수 있음

In [ ]:
from workalendar.asia import SouthKorea

cal = SouthKorea()
holiday = pd.DataFrame()
for y in range(2015, 2023) :
    holiday = pd.concat([holiday, pd.DataFrame(cal.holidays(y))], axis = 0)

holiday.columns = ['Date', 'holiday']
holiday['Date'] = pd.to_datetime(holiday['Date'])
holiday['holiday'] = 1
holiday

* 기존 데이터에 휴무일 정보 결합하기.
* 휴무일이 아닌 날짜는 0으로 저장하시오.

In [ ]:
df= pd.merge(df, holiday, how = 'left').fillna({'holiday':0})
# df = df.fillna({'holiday':0})
df.tail(30)

In [ ]:
df['holiday'] = df['holiday'].astype(int)

#### 3) 7일 이동평균 대기시간
* rolling().mean() 사용

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

df['WaitingTime(7days)'] = df['WaitingTime'].rolling(7).mean()

In [ ]:
df['WaitingTime(7days)'].head(10)

In [ ]:
df['WaitingTime(7days)'].fillna(df['WaitingTime(7days)'][6], inplace=True)

In [ ]:
df.head(10)

#### 4) 탑승률

In [ ]:
df['RidingPercent'] = (df['RidingCount'] / df['Reservation']) * 100

In [ ]:
df.head()

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.



## 4.데이터 저장
* **세부요구사항**
    * joblib 을 사용하여 작업 경로에 정리한 데이터프레임을 저장합니다.
        * 저장파일이름 : data1.pkl

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.

joblib.dump(df, 'data1.pkl')